## 经典数据表示

In [ ]:
浮点数（IEEE 754 normal/subnormal）


浮点数 (bit for sign)(bits for exponent)(bits for mantissa)的方式，来近似实数. 采用二进制，它在数值上相等的实数值 = sign * [1 + fraction] * 2^(magnitude)  
  
fraction 是用 bits for mantissa 计算出来的。bits for mantissa，比如 FP64 有 52 位bits, 二进制下这些 bits 的第1位到第52位分别代表 1/2 ,..., 1/2^52。累加后，加 1 (默认隐含)  
  
magnitude 是用 bits for exponent 计算出来的。bits for exponent，比如 FP64 有 11 位bits, 二进制下这些 bits 可算出一个十进制指数. 减去偏置 bias（根据具体类型确定）  


Q1：fraction 为什么要加 1?  
A1：加 1 ，使得最终数值等于 1.xxx * 2^E 的形式，如果没有加 1，那么最终数值等于 0.xxx * 2^E 的形式。说明 mantissa 省略了科学计数法底数里的 1，节省出来的bits扩大了表示范围  
  
Q2：magnitude 为什么要减 bias?  
A2：简而言之是为了表示出 负指数，用以表示小的数。比如 FP64 exponent部分有 11 位，按理最大(11个1)二进制表示2^11-1=2047。为了表示负指数，且正负平等，那么2047的一半1023作为bias，  
通过减去 bias 得到 负指数。所以最大的正指数是 2046-1023 = 1023（全1的2047在FP64里代表正无穷了），最小的负指数是 1-1023 = 1022（全0的exponent不符合normal规范, 会触发subnormal规范）

In [ ]:
它们都分成3部分，符号位，指数和尾数。不同精度只不过是指数位和尾数位的长度不一样。

解析一个浮点数就5条规则

如果指数位全零，尾数位是全零，那就表示0
如果指数位全零，尾数位是非零，就表示一个很小的数（subnormal），计算方式 (−1)^signbit × 2^−126 × 0.fractionbits
如果指数位全是1，尾数位是全零，表示正负无穷
如果指数位全是1，尾数位是非零，表示不是一个数NAN
剩下的计算方式为 (−1)^signbit × 2^(exponentbits − bias) × 1.fractionbits
常用的语言几乎都不提供半精度（英伟达2002年导入）的浮点数，这时候需要我们自己转化。

In [ ]:
FP64 双精度浮点 8bytes 64bits
1位符号+11位指数+52位小数
range: 10^-308 至 10^308，15-17位小数精度
IEEE 754定义，大多数C/C++系统里的double类型，x86CPU支持，GPU内支持但一般严重限制性能（除非一些专业卡）。一些科学计算（特别是迭代的）必须使用这个最高精度不然累积误差过大

In [ ]:
FP32 单精度 4bytes 32bits
1位符号+8位指数+23位小数
range: 10^-38 至 10^38，6-9位小数精度
IEEE 754定义，大多数C/C++系统里的float类型，x86CPU支持，GPU支持，是长期以来深度学习/神经网络的标准类型。
理想情况下，训练和推理中涉及的数字都应该用 FP32 精度。但由于性能问题，实践中通常使用混合精度：
训练：
    使用 FP32 精度的weights作为 精确的 master weights。
    使用 FP16/BF6 等精度的权重作前向计算、后向传播（即梯度是 FP16/BF16）
    使用 FP16/BF16 的梯度去更新 FP32 的主权重
推理：
    FP16 精度的weights即有不错的效果

In [ ]:
FP16 半精度 2bytes 16bits
1位符号+5位指数+10位小数
range: 10^-5 至 65504，4位小数精度
英伟达 定义，C/C++ 不支持，x86CPU 不支持，新GPU良好支持。容易上溢/下溢

In [ ]:
Int8